<a href="https://colab.research.google.com/github/thejasrao262003/CVIP-Data-Science/blob/main/Phase%202/Golden%20Task/speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
Root = "drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
!pip install "librosa==0.9.1"

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

<ipython-input-4-138437059bc8>:16: FutureWarning: Pass y=[ 3.0517578e-05  6.1035156e-05  3.0517578e-05 ... -2.0751953e-03
  8.2397461e-04 -6.7138672e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-4-138437059bc8>:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-4-138437059bc8>:16: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-4-138437059bc8>:16: FutureWarning: Pass y=[ 0.0000000e+00 -6.1035156e-05 -9.1552734e-05 .

In [ ]:
x_train

array([[-5.76583557e+02,  6.80707321e+01, -2.08497734e+01, ...,
         9.47496756e-06,  4.73085856e-06,  2.85588840e-06],
       [-7.09056824e+02,  5.57342987e+01,  2.66830969e+00, ...,
         2.82708083e-06,  2.46006584e-06,  1.46975708e-06],
       [-6.19052124e+02,  5.22264061e+01,  1.48722153e+01, ...,
         2.21770806e-05,  1.10033261e-05,  7.94161861e-06],
       ...,
       [-5.24454834e+02,  3.93770943e+01, -5.10801458e+00, ...,
         3.39247723e-04,  1.07432068e-04,  7.15327405e-05],
       [-3.82501648e+02,  3.61315613e+01, -1.87694397e+01, ...,
         1.95721514e-03,  1.30252587e-03,  6.14391174e-04],
       [-6.47903625e+02,  4.48624649e+01, -2.82318783e+00, ...,
         4.23921774e-05,  4.01656071e-05,  1.77828169e-05]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)
y_pred

array(['calm', 'calm', 'disgust', 'calm', 'disgust', 'disgust', 'disgust',
       'happy', 'calm', 'calm', 'calm', 'calm', 'disgust', 'calm',
       'fearful', 'disgust', 'calm', 'fearful', 'happy', 'disgust',
       'calm', 'happy', 'calm', 'calm', 'calm', 'disgust', 'disgust',
       'calm', 'happy', 'happy', 'disgust', 'calm', 'disgust', 'fearful',
       'disgust', 'fearful', 'disgust', 'disgust', 'disgust', 'fearful',
       'disgust', 'calm', 'happy', 'fearful', 'disgust', 'fearful',
       'fearful', 'fearful', 'disgust', 'happy', 'calm', 'happy',
       'fearful', 'happy', 'calm', 'disgust', 'calm', 'fearful',
       'disgust', 'calm', 'fearful', 'calm', 'fearful', 'disgust',
       'fearful', 'disgust', 'calm', 'fearful', 'disgust', 'fearful',
       'calm', 'fearful', 'calm', 'happy', 'fearful', 'calm', 'happy',
       'fearful', 'disgust', 'calm', 'fearful', 'calm', 'happy', 'happy',
       'fearful', 'calm', 'calm', 'disgust', 'calm', 'calm', 'calm',
       'fearful', 'fear

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 66.67%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.80392157, 0.62626263, 0.65306122, 0.56470588])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,disgust,calm
2,disgust,disgust
3,calm,calm
4,disgust,disgust
5,happy,disgust
6,happy,disgust
7,happy,happy
8,calm,calm
9,happy,calm
